In [1]:
import numpy as np
import pandas as pd
import random

In [2]:
def loadFile(filename, multi_col=False):
    contents = []
    with open(filename) as file:
        for line in file:
            temp = line.strip('\n').strip()
            if multi_col:
                temp = temp.split()
            contents.append(temp)
    
    return np.asarray([np.array(x) for x in contents])

In [3]:
prob_a1_df = loadFile('prob_a1.txt', multi_col=True).astype(np.float32)
prob_a2_df = loadFile('prob_a2.txt', multi_col=True).astype(np.float32)
prob_a3_df = loadFile('prob_a3.txt', multi_col=True).astype(np.float32)
prob_a4_df = loadFile('prob_a4.txt', multi_col=True).astype(np.float32)
rewards_df = loadFile('rewards.txt').astype(np.int32)

In [4]:
gamma = 0.9925

In [5]:
probs = np.zeros((81, 81, 4))

In [6]:
condensed_probs = [prob_a1_df, prob_a2_df, prob_a3_df, prob_a4_df]
for i in range(len(condensed_probs)):
    condensed_prob = condensed_probs[i]
    for prob in condensed_prob:
        probs[int(prob[0]) - 1][int(prob[1]) - 1][i] = prob[2]

In [22]:
valid_states = np.array([3, 11,12,15,16,17,20,22,23,24,26,29,30, 31, 34, 35, 39, 43, 48, 56, 57, 58, 59, 60, 61, 62, 66, 70, 71])
valid_states = valid_states - 1

In [7]:
def calcValFunc(policy):
    newProbMat = None
    for pi in range(policy.shape[0]):
        temp = probs[pi, :, policy[pi]]
        if newProbMat is None:
            newProbMat = temp
        else:
            newProbMat = np.vstack((newProbMat, temp))
    newProbMat = np.identity((newProbMat.shape[0])) - gamma * newProbMat
    
    return np.matmul(np.linalg.inv(newProbMat), rewards_df)

In [8]:
def calcNewValFunc(valueFunction, policy):
    newProbMat = None
    for pi in range(policy.shape[0]):
        temp = probs[pi, :, policy[pi]]
        if newProbMat is None:
            newProbMat = temp
        else:
            newProbMat = np.vstack((newProbMat, temp))
    return rewards_df + gamma * np.sum(newProbMat * valueFunction, axis=0)

In [9]:
def getNewPolicy(valueFunction, policy):
    for pi in range(policy.shape[0]):
        max_val = -np.inf
        max_action = -np.inf
        for action in range(4):
            temp = np.sum(probs[pi, :, action] * valueFunction)
            if max_val < temp:
                max_val = temp
                max_action = action
        policy[pi] = max_action
    return policy

In [23]:
def getRandomPolicy():
    policy = np.empty((81,), dtype=np.int32)
    for i in range(policy.shape[0]):
        policy[i] = random.randint(0, 3)
    return policy

In [24]:
policy = getRandomPolicy()
valueFunc = calcValFunc(policy)
opt_policy = getNewPolicy(valueFunc, policy)
newValueFunc = calcNewValFunc(valueFunc, opt_policy)#calcValFunc(opt_policy)#

In [25]:
i = 0
while not np.equal(newValueFunc, valueFunc).all():
    i += 1
    valueFunc = newValueFunc.copy()
    opt_policy = getNewPolicy(valueFunc, opt_policy)
    newValueFunc = calcValFunc(opt_policy)
print('Convergence in {} steps'.format(i))

Convergence in 5 steps


In [26]:
valueFunc 

array([   0.        ,    0.        ,  100.7010102 ,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ,  102.37529123,  101.52367323,
          0.        ,    0.        ,  109.48995052,  110.40904748,
        111.33585966,    0.        ,    0.        ,  103.23464898,
          0.        ,  106.77828777,  107.67464526,  108.57850454,
          0.        ,  112.27045183,    0.        ,    0.        ,
        104.10123631,  104.97509851,  105.88855749,    0.        ,
          0.        ,  114.16323791,  113.21288929,    0.        ,
          0.        ,    0.        ,  103.78143159,    0.        ,
          0.        ,    0.        ,  115.12156408,    0.        ,
          0.        ,    0.        , -133.33333333,   90.98540322,
       -133.33333333,    0.        , -133.33333333,  116.08793478,
        122.02491933,    0.        ,    0.        ,   81.39950233,
         93.67166485,   95.1728646 ,  108.34262576,  109.58365

In [27]:
def valueFuncIteration(valueFunction):
    newProbMat = None
    for pi in range(policy.shape[0]):
        max_val = -np.inf
        max_temp = None
        for a in range(4):
            temp = probs[pi, :, a]
            temp_val = np.sum(temp * valueFunction, axis=0)
            if max_val < temp_val:
                max_val = temp_val
                max_temp = temp
        if newProbMat is None:
            newProbMat = max_temp
        else:
            newProbMat = np.vstack((newProbMat, max_temp))
    return rewards_df + gamma * np.sum(newProbMat * valueFunction, axis=1)

In [28]:
policy = getRandomPolicy()
valueFunc2 = np.array([0]*policy.shape[0])
tempFunc = valueFuncIteration(valueFunc2)

In [29]:
i = 0
while np.sum(np.absolute(valueFunc2 - tempFunc)) > 1e-6:
    i += 1
    valueFunc2 = tempFunc
    tempFunc = valueFuncIteration(valueFunc2)
print('Convergence in {} steps'.format(i))

Convergence in 2304 steps


In [30]:
valueFunc2

array([   0.        ,    0.        ,  100.70100671,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ,  102.37528774,  101.52366974,
          0.        ,    0.        ,  109.48994702,  110.40904399,
        111.33585617,    0.        ,    0.        ,  103.23464549,
          0.        ,  106.77828428,  107.67464177,  108.57850104,
          0.        ,  112.27044834,    0.        ,    0.        ,
        104.10123282,  104.97509502,  105.888554  ,    0.        ,
          0.        ,  114.16323441,  113.2128858 ,    0.        ,
          0.        ,    0.        ,  103.78142811,    0.        ,
          0.        ,    0.        ,  115.12156059,    0.        ,
          0.        ,    0.        , -133.33332942,   90.98540012,
       -133.33332942,    0.        , -133.33332942,  116.08793128,
        122.02491563,    0.        ,    0.        ,   81.39949978,
         93.67166194,   95.17286168,  108.34262248,  109.58365

In [35]:
action_map = {0: 'Left', 1: 'Up', 2: 'Right', 3: 'Down'}

In [40]:
for pi in range(policy.shape[0]):
    if pi in valid_states:
        print("""State: {} | Action: {} | Policy Iteration Value: {} | Value Iteration Value: {}""".format(pi + 1, 
                                                                                                       action_map[opt_policy[pi]], 
                                                                                                       valueFunc[pi], 
                                                                                                       valueFunc2[pi]))

State: 3 | Action: Right | Policy Iteration Value: 100.7010102002196 | Value Iteration Value: 100.70100670774
State: 11 | Action: Right | Policy Iteration Value: 102.37529123178209 | Value Iteration Value: 102.3752877393026
State: 12 | Action: Up | Policy Iteration Value: 101.52367322965442 | Value Iteration Value: 101.52366973717493
State: 15 | Action: Down | Policy Iteration Value: 109.48995051690076 | Value Iteration Value: 109.48994702413327
State: 16 | Action: Down | Policy Iteration Value: 110.40904747912874 | Value Iteration Value: 110.40904398636131
State: 17 | Action: Right | Policy Iteration Value: 111.33585966247094 | Value Iteration Value: 111.33585616970353
State: 20 | Action: Right | Policy Iteration Value: 103.23464897839825 | Value Iteration Value: 103.23464548591886
State: 22 | Action: Down | Policy Iteration Value: 106.7782877691457 | Value Iteration Value: 106.77828427637834
State: 23 | Action: Down | Policy Iteration Value: 107.6746452597115 | Value Iteration Value: